In [3]:
import pandas as pd
import numpy as np
import time 
import pickle
import os
#nltk.download('punkt')

from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import warnings 
warnings.filterwarnings("ignore")

import tensorflow as tf
from tensorflow import keras 
from tensorflow.keras.preprocessing.text import Tokenizer

2023-04-16 12:50:37.521595: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
#import pre-processed data from pickle 
df_10= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2010_final_rnn.pickle")
#df_09= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2009_text_wo_names.pickle")
#df_11= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2011_text_wo_names.pickle")
#df_12= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2012_text_wo_names.pickle")
#df_13= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2013_text_wo_names.pickle")
#df_14= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2014_text_wo_names.pickle")
#df_15= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2015_text_wo_names.pickle")
#df_16= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2016_text_wo_names.pickle")
#df_17= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2017_text_wo_names.pickle")
#df_18= pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2018_text_wo_names.pickle")
#df_19 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2019_text_wo_names.pickle")
#df_20 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2020_text_wo_names.pickle")
#df_21 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2021_text_wo_names.pickle")
#df_22 = pd.read_pickle(r"/Users/yolandaferreirofranchi/Desktop/ThesisDatasets/2022_text_wo_names_(1).pickle")
df_10

,pre_processed_sent,string_rnn,male_count,female_count,Proper_noun_list,pn exists,sentences,article_id,year,col_type
0,"[say, delight, restored, bridge, back, use]",say delight restored bridge back use,1,0,[Southease],None,"Chairman of Southease Parish, Neville Harrison...",1,2010,0
1,"[family, year, old, kill, house, fire, pay, tr...",family year old kill house fire pay tribute br...,0,2,[],None,The family of a 34-year-old mother from Bristo...,21,2010,1
2,"[family, say, kind, totally, dedicated]",family say kind totally dedicated,1,4,[],None,Her family said she was kind and a totally ded...,21,2010,1
3,"[truly, tragic, love, family, everything, give...",truly tragic love family everything give famil...,0,4,[Sara],None,"'Truly tragic'""Sara loved her family above eve...",21,2010,1
4,"[everybody, know, love, miss, always]",everybody know love miss always,0,3,[],None,"""Everybody who knew her will love her and miss...",21,2010,1
...,...,...,...,...,...,...,...,...,...,...
514542,"[contrast, news, conference, sound, guard, eve...",contrast news conference sound guard even slig...,2,0,[Fabio],None,Compare and contrast - Fabio Capello's news co...,2175804,2010,0
514543,"[goal, one, chosen, five, could, take, spot, k...",goal one chosen five could take spot kick need,2,0,"[Nelson, Mandela]",True,And after his goal at the Nelson Mandela Bay S...,2175804,2010,0
514544,"[everyone, practise, say]",everyone practise say,1,0,[],None,"""Everyone has been practising them,"" he said.",2175804,2010,0
514545,"[look, relax, hair, back, cornrows, mobile, tu...",look relax hair back cornrows mobile tuck righ...,4,0,[],None,"He looked relaxed, his hair back in cornrows, ...",2175804,2010,0


In [5]:
def absolute_count(male_col, female_col):
    if female_col > male_col and male_col == 0:
        return 1
    elif male_col> female_col and female_col ==0: 
        return 0
    else: 
        return None

In [6]:
#apply function to only get rows with an absolute count 
df_10['col_type'] = df_10.apply(lambda row: absolute_count(row['male_count'], row['female_count']),axis=1)

#remove nulls 
df_10 = df_10[df_10["col_type"].notnull()]

#DOC: number of male and female columns
df_10["col_type"].value_counts()  

0.0    378476
1.0    113870
Name: col_type, dtype: int64

In [7]:
# Split the df by the preprocessed 
x_train, x_test, y_train, y_test = train_test_split(df_10["pre_processed_sent"], 
                                                    df_10["col_type"], 
                                                    stratify = df_10["col_type"])

In [8]:

xTrain, xTest, yTrain, yTest = train_test_split(df_10["string_rnn"],
                                                df_10["col_type"], 
                                                stratify = df_10["col_type"])

In [34]:

train_dataset = tf.data.Dataset.from_tensor_slices((xTrain, yTrain)) #string_rnn here 
test_dataset = tf.data.Dataset.from_tensor_slices((xTest, yTest)) #clean text 
train_dataset

<TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.float64, name=None))>

In [10]:
for example, gen_label in train_dataset.take(1):
  print('text: ', example.numpy())
  print('label: ', gen_label.numpy())

text:  b'injury road cause physical emotional damage victim family also cost country dear economic term say death'
label:  0.0


In [11]:

buffer_size = 10000 #take first 10,000 tweets
batch_size = 64 #batch size that leads to highest accuracy (best practice)
train_dataset = train_dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [12]:
#map text features to integer sequences for all of the vocabulary
vocab_size = 100000
encoder = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size)
encoder.adapt(train_dataset.map(lambda text, label: text))

In [14]:
model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(len(encoder.get_vocabulary()), 64, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,  return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(32)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(10)
])

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          2753600   
                                                                 
 bidirectional (Bidirectiona  (None, None, 128)        66048     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 64)               41216     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                        

In [26]:
#EGGGGG
sample_text = ('went to the candy story'
               'attended baseball came with friends')
predictions = model.predict(np.array([sample_text]))
print(predictions[0])

1/1 [==============================] - 2s 2s/step
[ 1.1166545e-02 -1.4210087e-03  1.0205791e-03  3.5288846e-05
  4.8872638e-03 -7.3733656e-03 -1.3958568e-02 -5.5539357e-03
 -1.6895628e-03  2.7929714e-03]


In [20]:
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), #maybe change to BinaryCrossentropy
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [21]:
history = model.fit(train_dataset, epochs=3,
                    validation_data=test_dataset)

Epoch 1/3
5770/5770 [==============================] - 197s 33ms/step - loss: 0.5703 - accuracy: 0.7715 - val_loss: 0.4863 - val_accuracy: 0.7818
Epoch 2/3
5770/5770 [==============================] - 206s 36ms/step - loss: 0.4663 - accuracy: 0.7931 - val_loss: 0.4679 - val_accuracy: 0.7921
Epoch 3/3
5770/5770 [==============================] - 223s 39ms/step - loss: 0.4438 - accuracy: 0.8053 - val_loss: 0.4674 - val_accuracy: 0.7901


In [44]:
##THIS IS A WIP
# Import the necessary libraries
from sklearn.metrics import classification_report, confusion_matrix

y_pred = np.concatenate(list(map(lambda x: np.argmax(model.predict(x[0]), axis=-1)[..., np.newaxis], test_dataset)))
y_true = np.concatenate(list(map(lambda x: np.argmax(x[1], axis=-1)[..., np.newaxis], test_dataset)))


# Generate the confusion matrix
cm = confusion_matrix(y_true, y_pred)
print(cm)

IndexError: tuple index out of range

In [2]:
# Get the feature weights
weights = model.layers[1].get_weights()[0]

# Sort the feature weights
sorted_weights = np.sort(weights, axis=0)[::-1]

# Select the top 1000 features
top_features = sorted_weights[:1000]

# Select the bottom 1000 features
bottom_features = sorted_weights[-1000:]

# Map the features back to words
#top_words = [word for word, idx in sorted(vocab.items(), key=lambda x: weights[x[1]], reverse=True)[:1000]]
#bottom_words = [word for word, idx in sorted(vocab.items(), key=lambda x: weights[x[1]])[:1000]]
top_words = vocab[np.argsort(weights[:,::-1])[:1000]]
bottom_words = vocab[np.argsort(weights)][:1000]

# Print the top and bottom features
#print("Top 1000 features:", top_words)
#print("Bottom 1000 features:", bottom_words)

print("Top 1000 words:")
for word, weight in zip(top_words, top_features):
    coef_tuples = list(zip(word, weight))
    df_high = pd.DataFrame(coef_tuples, columns = ["word", "weight"])
    print(len(df_high))

NameError: name 'model' is not defined

In [1]:
df_high

NameError: name 'df_high' is not defined